In [7]:
import pandas as pd
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import splat
import numpy
import astropy.units as u
from astropy import constants as const 
import copy
from scipy.integrate import trapz        # for numerical integration
from scipy.interpolate import interp1d
from astropy.coordinates import SkyCoord, CylindricalDifferential
import pandas as pd
%matplotlib inline

In [8]:
DATA_FOLDER='/users/caganze/research/J1624/data/'

In [9]:
#nearby M dwarfs
df=pd.read_excel(DATA_FOLDER+'UCD_lateM_dwarf_precision_RV_20pc_thin_disk_population.xlsx')

In [10]:
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.coordinates import FK5

def compute_uvw_velocity(ra_J2000, dec_J2000, parallax, rv, mu_ra, mu_dec, e_parallax, e_rv, e_mu_ra, e_mu_dec, correct_lsr=True):
	"""
	Compute the Galactic UVW space velocity based on the formulation in Johnson and Soderblom (1987).
	Parameters
	----------
	ra 			:	float
					RA of the source in degrees
	dec 		:	float
					Dec of the source in degrees
	parallax	:	float
					the parallax in mas
	rv			:	float
					the radial velocity in km/s
	mu_ra		:	float
					the proper motion in right ascension in mas/yr
	mu_dec		:	float
					the proper motion in declination in mas/yr
	e_parallax	:	float
					the error of parallax in mas
	e_rv		:	float
					the error of radial velocity in km/s
	e_mu_ra		:	float
					the error of proper motion in right ascension in mas/yr
	e_mu_dec	:	float
					the error of proper motion in declination in mas/yr
	Optional Parameters
	-------------------
	correct_lsr	:	bool
					If True: uvw corrected to the LSR
	Returns
	-------
	uvw 		:	array-like
					UVW velocities in km/s
	e_uvw 		:	array-like
					errors of UVW velocities in km/s
	"""
	## convert proper motions and parallax from mas to arcsec
	parallax   /= 1000
	mu_ra 	   /= 1000
	mu_dec 	   /= 1000

	e_parallax /= 1000
	e_mu_ra    /= 1000
	e_mu_dec   /= 1000

	## convert ra and dec into radians (the paper uses equinox 1950)
	coord_J2000 = SkyCoord(ra_J2000*u.deg, dec_J2000*u.deg, unit='deg', frame='icrs')

	coord_J1950 = coord_J2000.transform_to(FK5(equinox='J1950.0'))

	ra          = coord_J1950.ra.value
	dec         = coord_J1950.dec.value

	## degree to radian conversion
	deg_to_rad  = np.pi/180

	## define the A matrix
	A_ra      = np.array([	[	+np.cos(ra*deg_to_rad),		+np.sin(ra*deg_to_rad),	0],
							[	+np.sin(ra*deg_to_rad),		-np.cos(ra*deg_to_rad),	0],
							[						 0,							0, -1]])

	A_dec	  = np.array([	[	+np.cos(dec*deg_to_rad),	 0,	-np.sin(dec*deg_to_rad)],
							[						  0, 	-1,						  0],
							[	-np.sin(dec*deg_to_rad),	 0,	-np.cos(dec*deg_to_rad)]])

	A         = A_ra.dot(A_dec)

	#A0 		= np.array([[ 	+np.cos(ra*deg_to_rad)*np.cos(dec*deg_to_rad), -np.sin(ra*deg_to_rad), -np.cos(ra*deg_to_rad)*np.sin(dec*deg_to_rad)],
	#					[	+np.sin(ra*deg_to_rad)*np.cos(dec*deg_to_rad), +np.cos(ra*deg_to_rad), -np.sin(ra*deg_to_rad)*np.sin(dec*deg_to_rad)],
	#					[	+np.sin(dec*deg_to_rad) 					 , 				   	    0, +np.cos(dec*deg_to_rad)					   ]])

	## define RA and Dec for the North Galactic Pole (NGP) in degrees
	ra_ngp  = 192.25
	dec_ngp = 27.4
	theta0  = 123 # the position angle of NGP relative to great semi-circle of the North Celetial Pole and the zero Galactic longitude
	
	T1      = np.array([[  +np.cos(theta0*deg_to_rad), +np.sin(theta0*deg_to_rad),  0],
						[  +np.sin(theta0*deg_to_rad), -np.cos(theta0*deg_to_rad),  0],
						[  						    0,			 			 	0,  +1]])

	T2      = np.array([[-np.sin(dec_ngp*deg_to_rad),  0, +np.cos(dec_ngp*deg_to_rad)],
						[						   0, -1, 						  	0],
						[+np.cos(dec_ngp*deg_to_rad),  0, +np.sin(dec_ngp*deg_to_rad)]])

	T3      = np.array([[  +np.cos(ra_ngp*deg_to_rad), +np.sin(ra_ngp*deg_to_rad),  0],
						[  +np.sin(ra_ngp*deg_to_rad), -np.cos(ra_ngp*deg_to_rad),  0],
						[						    0,						    0, +1]])

	## define the T matrix
	T       = T1.dot(T2.dot(T3))

	## B matrix = TA
	B       = T.dot(A)

	## uvw matrix
	k       = 1.4959787 * 10**8 / 365.24219879 / 24 /3600 #4.74057 # AU/tropical yr (km/s)
	uvw     = B.dot(np.array([	[rv], 
								[k * mu_ra 	/ parallax], 
								[k * mu_dec / parallax]]))

	## solar uvw from Schonrich et al. (2010)
	uvw_solar = np.array([	[11.1],	[12.24], [7.25]	])

	C       = B**2
	e_uvw2  = C.dot(np.array([	[ e_rv**2], 
								[ (k/parallax)**2 * ( e_mu_ra**2  + ( mu_ra  * e_parallax / parallax )**2 )], 
								[ (k/parallax)**2 * ( e_mu_dec**2 + ( mu_dec * e_parallax / parallax )**2 )]	])) \
					+ 2 * mu_ra * mu_dec * k**2 * e_parallax**2 / parallax**4 * \
					np.array([ 	[ B[0][1]*B[0][2] ], 
								[ B[1][1]*B[1][2] ], 
								[ B[2][1]*B[2][2] ] ])

	if correct_lsr: uvw += uvw_solar

	return uvw, np.sqrt(e_uvw2)

In [224]:
def compute_pm_from_uvw(ra_J2000, dec_J2000, parallax, uvw, correct_lsr=True):
    """
    Compute the Galactic UVW space velocity based on the formulation in Johnson and Soderblom (1987).
    Parameters
    ----------
    ra 			:	float
                    RA of the source in degrees
    dec 		:	float
                    Dec of the source in degrees
    parallax	:	float
                    the parallax in mas
    rv			:	float
                    the radial velocity in km/s
    mu_ra		:	float
                    the proper motion in right ascension in mas/yr
    mu_dec		:	float
                    the proper motion in declination in mas/yr
    e_parallax	:	float
                    the error of parallax in mas
    e_rv		:	float
                    the error of radial velocity in km/s
    e_mu_ra		:	float
                    the error of proper motion in right ascension in mas/yr
    e_mu_dec	:	float
                    the error of proper motion in declination in mas/yr
    Optional Parameters
    -------------------
    correct_lsr	:	bool
                    If True: uvw corrected to the LSR
    Returns
    -------
    uvw 		:	array-like
                    UVW velocities in km/s
    e_uvw 		:	array-like
                    errors of UVW velocities in km/s
    """
    ## convert proper motions and parallax from mas to arcsec
    parallax   /= 1000
    #mu_ra 	   /= 1000
    #mu_dec 	   /= 1000

    #e_parallax /= 1000
    #e_mu_ra    /= 1000
    #e_mu_dec   /= 1000

    ## convert ra and dec into radians (the paper uses equinox 1950)
    coord_J2000 = SkyCoord(ra_J2000*u.deg, dec_J2000*u.deg, unit='deg', frame='icrs')

    coord_J1950 = coord_J2000.transform_to(FK5(equinox='J1950.0'))

    ra          = coord_J1950.ra.value
    dec         = coord_J1950.dec.value

    ## degree to radian conversion
    deg_to_rad  = np.pi/180

    ## define the A matrix
    A_ra      = np.array([	[	+np.cos(ra*deg_to_rad),		+np.sin(ra*deg_to_rad),	0],
                            [	+np.sin(ra*deg_to_rad),		-np.cos(ra*deg_to_rad),	0],
                            [						 0,							0, -1]])

    A_dec	  = np.array([	[	+np.cos(dec*deg_to_rad),	 0,	-np.sin(dec*deg_to_rad)],
                            [						  0, 	-1,						  0],
                            [	-np.sin(dec*deg_to_rad),	 0,	-np.cos(dec*deg_to_rad)]])

    A         = A_ra.dot(A_dec)

    #A0 		= np.array([[ 	+np.cos(ra*deg_to_rad)*np.cos(dec*deg_to_rad), -np.sin(ra*deg_to_rad), -np.cos(ra*deg_to_rad)*np.sin(dec*deg_to_rad)],
    #					[	+np.sin(ra*deg_to_rad)*np.cos(dec*deg_to_rad), +np.cos(ra*deg_to_rad), -np.sin(ra*deg_to_rad)*np.sin(dec*deg_to_rad)],
    #					[	+np.sin(dec*deg_to_rad) 					 , 				   	    0, +np.cos(dec*deg_to_rad)					   ]])

    ## define RA and Dec for the North Galactic Pole (NGP) in degrees
    ra_ngp  = 192.25
    dec_ngp = 27.4
    theta0  = 123 # the position angle of NGP relative to great semi-circle of the North Celetial Pole and the zero Galactic longitude

    T1      = np.array([[  +np.cos(theta0*deg_to_rad), +np.sin(theta0*deg_to_rad),  0],
                        [  +np.sin(theta0*deg_to_rad), -np.cos(theta0*deg_to_rad),  0],
                        [  						    0,			 			 	0,  +1]])

    T2      = np.array([[-np.sin(dec_ngp*deg_to_rad),  0, +np.cos(dec_ngp*deg_to_rad)],
                        [						   0, -1, 						  	0],
                        [+np.cos(dec_ngp*deg_to_rad),  0, +np.sin(dec_ngp*deg_to_rad)]])

    T3      = np.array([[  +np.cos(ra_ngp*deg_to_rad), +np.sin(ra_ngp*deg_to_rad),  0],
                        [  +np.sin(ra_ngp*deg_to_rad), -np.cos(ra_ngp*deg_to_rad),  0],
                        [						    0,						    0, +1]])

    ## define the T matrix

    T       = T1 @ T2 @ T3


    ## B matrix = TA
    B       = T @ A
    uvw_solar = np.array([	[11.1],	[12.24], [7.25]	])

    if correct_lsr: uvw += uvw_solar
        
    print (np.vstack(B).T)
    
    Bm= B.reshape(-1, B.shape[0], B.shape[-1])
    
    print (np.shape(Bm), np.shape(uvw))
    motion=np.linalg.solve(Bm,uvw)

    ## uvw matrix
    k       = 1.4959787 * 10**8 / 365.24219879 / 24 /3600 #4.74057 # AU/tropical yr (km/s)
    rv=motion[:,0]
    mu_ra=motion[:,1]/(k*parallax)
    mu_dec=motion[:,-1]/(k*parallax)
    vtan=np.sqrt(k* (mu_ra**2+ mu_dec**2))*parallax
    return np.array([rv, mu_ra, mu_dec, vtan])

In [233]:
#double-check for trappist one
trap= SkyCoord(ra=346.6250957*u.deg, dec=-5.0428081*u.deg,  
               radial_velocity=-51*u.km/u.s,  pm_ra_cosdec=(922.0*u.mas/u.yr)*np.cos(-5.0428081*u.deg.to(u.radian)),
               pm_dec=-471.9*u.mas/u.yr, distance=12.49*u.pc)
#proper_motion_to_uvw(trap, 922.0*1e-3, 471.9*1e-3, -51.688, 80.09*1e-3
from astropy.coordinates import (CartesianRepresentation,CartesianDifferential)


In [234]:
compute_uvw_velocity(346.6250957, -5.0428081, 80.09,
                     -51, 922.0, -471.9,   0.0,  0.0, 0.0, 0.0, correct_lsr=False)

(array([[-43.74782869],
        [-65.85697762],
        [ 10.41498636]]),
 array([[0.],
        [0.],
        [0.]]))

In [155]:
ras=np.ones(10)*346.6250957
decs=np.ones(10)*-5.0428081
ds=np.ones(10)*12.49
ux=np.ones(10)*-43.74782869
vx=np.ones(10)*-65.85697762
wx=np.ones(10)*10.41498636

In [156]:
np.shape(A_list), np.shape(B_list)

((10, 2, 2), (10, 2))

In [15]:
import popsims

In [157]:
vel=np.array([[-43.74782869, -65.85697762, 10.41498636]])

In [185]:
np.shape(np.vstack([ux, vx, wx]))[-1]

10

In [235]:
compute_pm_from_uvw(346.6250957, -5.0428081,12.49, vel, correct_lsr=False)

[[ 0.19060852  0.51568714 -0.83530543]
 [-0.86298271 -0.31754911 -0.39296745]
 [-0.46789875  0.79575708  0.38450152]]
(1, 3, 3) (1, 3)


array([[ -51.        ],
       [ 921.7012766 ],
       [-471.74710682],
       [  28.15730639]])

In [236]:
#A_list

In [237]:
#B_list

In [205]:
Bm=np.array([[ux], [vx], [wx]])

In [238]:
#Bm.T

In [239]:
np.shape(A_list), np.shape(Bm.T), np.shape(B_list), np.shape(np.vstack([ux, vx, wx]).T)

((10, 2, 2), (10, 1, 3), (10, 2), (10, 3))

In [250]:
#

In [248]:
compute_pm_from_uvw(ras[0], decs[0], ds[0], \
                   vel, correct_lsr=False)

[[ 0.19060852  0.51568714 -0.83530543]
 [-0.86298271 -0.31754911 -0.39296745]
 [-0.46789875  0.79575708  0.38450152]]
(1, 3, 3) (1, 3)


array([[-5.10000000e+01],
       [ 9.21701277e+50],
       [-4.71747107e+50],
       [ 2.81573064e+01]])

In [64]:
ras[0], decs[0], ds[0], ux[0], vs[0], ws[0],

(346.6250957,
 -5.0428081,
 1.2489999999999999e-05,
 -43.74782869,
 -65.85697762,
 10.41498636)

In [87]:
np.random.seed(11)
k = 10
A_list = np.random.rand(k,2,2)
B_list = np.random.rand(k,2)
solution = np.linalg.solve(A_list,B_list)

In [97]:
B_list

array([[0.06368643, 0.36461564],
       [0.0700228 , 0.31936771],
       [0.0703826 , 0.29026367],
       [0.79010112, 0.90540032],
       [0.79262139, 0.56181871],
       [0.61601839, 0.36148354],
       [0.1688173 , 0.43624093],
       [0.73282534, 0.06288762],
       [0.02073298, 0.77054807],
       [0.29995201, 0.70116428]])

In [119]:
A_list

array([[[0.18026969, 0.01947524],
        [0.46321853, 0.72493393]],

       [[0.4202036 , 0.4854271 ],
        [0.01278081, 0.48737161]],

       [[0.94180665, 0.85079509],
        [0.72996447, 0.10873607]],

       [[0.89390417, 0.85715425],
        [0.16508662, 0.63233401]],

       [[0.02048361, 0.11673727],
        [0.31636731, 0.15791231]],

       [[0.75897959, 0.81827536],
        [0.34462449, 0.3187988 ]],

       [[0.11166123, 0.08395314],
        [0.71272594, 0.5995434 ]],

       [[0.05567368, 0.47979728],
        [0.40167648, 0.847979  ]],

       [[0.71784918, 0.60206405],
        [0.55238382, 0.9491024 ]],

       [[0.98667333, 0.33805405],
        [0.23987468, 0.79643575]]])

In [120]:
np.shape(A_list)

(10, 2, 2)

In [ ]:

#rv_unc=0.13

#rvs=np.random.normal(0.3, 0.13, 1000)

#ms=np.array([proper_motion_to_uvw(coord, x, -25.81/100, -185.78) for x in rvs])

In [ ]:
tarvx, tarvy, tarvz

In [ ]:
compute_uvw_velocity(

In [ ]:
tarvx, tarvy, tarvz

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import Angle
result = Vizier.query_region(coord, radius=Angle(2, "arcsec"), catalog='Gaia')

In [ ]:
result[2]

In [ ]:
result[2]['Plx'][0]

In [ ]:
compute_uvw_velocity(result[2]['RA_ICRS'][0], result[2]['DE_ICRS'][0], 
                     result[2]['Plx'][0], 0.23, result[2]['pmRA'][0], result[2]['pmDE'][0], 
                     result[2]['e_Plx'][0], 
                     0.07,  result[2]['e_pmRA'][0],  result[2]['e_pmDE'][0], correct_lsr=True)

In [ ]:
coord=SkyCoord(ra=result[2]['RA_ICRS'][0]*u.deg, dec=result[2]['DE_ICRS'][0]*u.deg)
tarvx, tarvy, tarvz=np.array(proper_motion_to_uvw(coord,  result[2]['pmRA'][0], result[2]['pmDE'][0],
                                                  0.23, result[2]['Plx'][0])).flatten()
print(tarvx, tarvy, tarvz)

In [ ]:
#kiman catalog
#kiman catalog
data=pd.read_hdf(DATA_FOLDER+'/merged_Mdwarfs.h5', key='merged')

In [ ]:
mask=np.logical_and.reduce([data.photometric_sample_subg !=0,
                            data.photometric_sample_subred !=0,
                            data.GOODPHOT_SDSS ==1, 
                            data.GOODMATCH==1,
                            abs(data.parallax_error/data.parallax)<0.2,
                           data.SPT_x <8.])

In [ ]:
fig, ax=plt.subplots()

plt.scatter(data.V[mask], ((data.U**2+data.W)**0.5)[mask], s=1, alpha=0.4, label='Kiman Ms')
#plt.scatter(data.VZ, (data.VX**2+data.VY)**0.5, s=1, alpha=0.4, label='Kiman Ms')
plt.scatter(df.V, (df.U**2+df.W**2)**0.5, s=100, marker='+', label='Late Ms')
ax.errorbar(tarvz, (tarvx**2+ tarvy**2)**0.5, ms=20,  fmt='*', color='k')
plt.xlim([-200, 200])
plt.ylim([-1, 200])
plt.xlabel('V', fontsize=16)
plt.ylabel(r'$(U^2+ W^2)^{0.5}$', fontsize=16)
plt.legend(loc='lower left')


In [ ]:
from astroquery.vizier import Vizier
Vizier.ROW_LIMIT = -1 
gcs=Vizier.get_catalogs('J/A+A/530/A138')


In [ ]:
metal=(0.29, 0.07)

In [ ]:
metal

In [ ]:

gcs_df=gcs[0].to_pandas()

In [ ]:
gcs_bools=gcs_df['__Fe_H_'].between(metal[0]-3*metal[-1], metal[0]+3*metal[-1] )

In [ ]:
metal[0]-3*metal[-1],

In [ ]:
fig, ax=plt.subplots()
plt.scatter( gcs_df['ageMLP'],gcs_df['__a_Fe_'], c=gcs_df['__Fe_H_'], s=10, marker='+')

In [ ]:
fig, ax=plt.subplots()
#h=ax.hist(gcs[0]['ageMLP'], histtype='step',  lw=3.5, density=True)
h=ax.hist(gcs_df['ageMLP'][gcs_bools],  histtype='step',  lw=3.5, density=True, 
         label='GCS ')
plt.xlabel('')

In [ ]:
#plt.plot(gcs[0]['__Fe_H_'], gcs[0]['age'])

In [ ]:
#gala
import gala.coordinates as gc
import gala.dynamics as gd
import gala.potential as gp
import popsims
from gala.units import galactic
from loki import loki

In [ ]:
p= popsims.Pointing(coord=coord, name='J1624') 

In [ ]:
#np.arccos?

In [ ]:
#generate distances along this line of sight
#ds=p.draw_distances(0.1, 2000, 350, nsample=1e4)
NPOINTS=int(1e3)
TIMES=int(1e3)
ds=np.random.uniform(0, 2000, NPOINTS)
ux= np.random.uniform(0, 1, NPOINTS)
v=np.random.uniform(0, 1, NPOINTS)
rand_coord=SkyCoord(l=2*np.pi*ux*u.radian, 
                    b=(np.arccos(2*v-1)-np.pi/2)*u.rad, frame='galactic')

In [ ]:
plt.scatter(rand_coord.galactic.l.radian, rand_coord.galactic.b.radian, s=1)

In [ ]:
from astropy.coordinates import Galactic

In [ ]:
#from astropy.coordinates import CartesianRepresentation,
#CartesianDifferential, SphericalRepresentation, CylindricalRepresentation
                                 


In [ ]:
#gl.frame_specific_representation_info

In [ ]:
#create new coord object
new_coord= SkyCoord(ra=rand_coord.icrs.ra, dec=rand_coord.icrs.dec, distance=ds*u.pc,
                    radial_velocity=np.random.uniform(-100, 100,len(ds))*u.km/u.s, 
                    pm_dec=np.random.uniform(-100, 100,len(ds))*u.mas/u.yr, 
                    pm_ra_cosdec=np.random.uniform(-100, 100,len(ds))*u.mas/u.yr, 
                    frame='icrs')  


In [ ]:
import astropy.coordinates as astrocoord

In [ ]:
new_coord.representation_type

In [ ]:
#new_coord.transform_to(astrocoord.Galactocentric).galactic.velocity

In [ ]:
#m[1]
milky_way = gp.MilkyWayPotential()
H = gp.Hamiltonian(milky_way)
w0 = gd.PhaseSpacePosition(new_coord.cartesian)
orbits=H.integrate_orbit(w0,t=np.logspace(0, 2,TIMES)*u.Myr, cython_if_possible=True)


In [ ]:
vx, vy, vz=orbits.v_xyz

In [ ]:
(10**4.2)*u.Myr.to(u.Gyr)

In [ ]:
fig, ax=plt.subplots()
rs=(orbits.cartesian.x**2+orbits.cartesian.y**2)**0.5
for idx in np.random.choice(np.arange(0, NPOINTS-1), int(NPOINTS/100)):
    h=plt.scatter(rs[:,idx], orbits.cartesian.z[:,idx], 
               s=1, alpha=.1, marker='+', c=np.log10(orbits.t.value))
    plt.xlabel('R')
    plt.ylabel('Z')

In [ ]:
#rp=new_coord.represent_as(CylindricalRepresentation)

In [ ]:
#rp.to_cartesian()

In [ ]:
mean_vz=np.nanmedian(vz.to(u.km/u.s).value, axis=1)
std_vz=vz.to(u.km/u.s).std(axis=1).to(u.km/u.s).value
mean_z = np.nanmedian(orbits.cartesian.z.to(u.pc).value, axis=1)
std_z=np.nanstd(orbits.cartesian.z.to(u.pc).value, axis=1)

In [ ]:
weights= np.abs(orbits.cartesian.z.to(u.pc).value) <20

In [ ]:
weighted_vz= np.nanmedian(vz.to(u.km/u.s).value*weights, axis=1)
weighted_z= np.nanmedian(orbits.cartesian.z.to(u.pc).value*weights, axis=1)

In [ ]:
total_time=weights.T * np.reshape(orbits.t.value, ( TIMES))

In [ ]:
max_time=total_time.max(axis=1)*u.Myr.to(u.Gyr)#.value

In [ ]:
#total_time

In [ ]:
from scipy import stats

In [ ]:
#width = smax - smin
x = np.linspace(max_time.min(),  max_time.max(), 100)
y = stats.gaussian_kde(max_time)(x)

In [ ]:
fig, ax=plt.subplots()
#for idx in np.random.choice(np.arange(0, 9000), 5000):
h=ax.hist(max_time, bins='auto', log=True, histtype='step', density=True)
plt.plot(x, y, label='Gaussian Kernel')
#ax.fill_between(orbits.t.value, mean_z - std_z, mean_z + std_z, alpha=0.2, 
#                color='r')
#ax.axhline(coord.cartesian.z.to(u.pc).value)

plt.xlabel('t within 20 pc (Myr)')
#plt.ylabel(r' <|Z|> (pc)')
#plt.xscale('log')

In [ ]:
np.save( DATA_FOLDER+'/interp_20pc_sim.npy', np.array([x, y]))

In [ ]:
coord.cartesian.represent_as(CartesianDifferential)

In [ ]:
from galpy.orbit import Orbit
from galpy.potential import MWPotential2014

In [ ]:
o=Orbit(new_coord)

In [ ]:
#ts=np.linspace(0, 10**4.1,TIMES)#*u.Myr, 
#o.integrate(ts,MWPotential2014)

In [ ]:
o=orbits.to_galpy_orbit()
#p=o.plot3d(alpha=0.4)
#>>> xlim(-100.,100.)
#>>> ylim(-100.,100)
#>>> gca().set_zlim3d(-100.,100);

In [ ]:
#o.integrate?

In [ ]:
import galpy

In [ ]:
%matplotlib notebook

In [ ]:
#

In [ ]:
#x

In [ ]:
 np.sort(np.random.choice(len(x), 10))

In [ ]:
import astropy.units as u